In [1]:
# Imports

import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import librosa
from tqdm import tqdm
from scipy.io import wavfile as wav

import tensorflow as tf
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense,Dropout,Activation,Flatten
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import ModelCheckpoint

from sklearn import metrics
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split

In [2]:
label_dict = {0 : 'air conditioner',
              1 : 'car horn', 
              2 : 'children playing', 
              3 : 'dog bark', 
              4 : 'drilling', 
              5 : 'engine idling',
              6 : 'gun shot',
              7 : 'jackhammer',
              8 : 'siren',
              9 : 'street music'
}

audio_dataset_path='UrbanSound8K/audio/'
metadata=pd.read_csv('UrbanSound8K/metadata/UrbanSound8K.csv')

In [4]:
def features_extractor(file):
    audio, sample_rate = librosa.load(file_name, res_type='kaiser_fast') 
    mfccs_features = librosa.feature.mfcc(y=audio, sr=sample_rate, n_mfcc=40)
    mfccs_scaled_features = np.mean(mfccs_features.T, axis=0)
    
    return mfccs_scaled_features

In [5]:
extracted_features=[]

for index_num, row in tqdm(metadata.iterrows()):
    file_name = os.path.join(os.path.abspath(audio_dataset_path), 'fold'+str(row["fold"])+'/', str(row["slice_file_name"]))
    final_class_labels = row["class"]
    data = features_extractor(file_name)
    extracted_features.append([data, final_class_labels])
    

3554it [02:55, 21.32it/s]/Users/dominikzeman/opt/anaconda3/lib/python3.9/site-packages/librosa/core/spectrum.py:222: UserWarning: n_fft=2048 is too small for input signal of length=1323
  warnings.warn(
8325it [06:38, 27.30it/s]/Users/dominikzeman/opt/anaconda3/lib/python3.9/site-packages/librosa/core/spectrum.py:222: UserWarning: n_fft=2048 is too small for input signal of length=1103
  warnings.warn(
/Users/dominikzeman/opt/anaconda3/lib/python3.9/site-packages/librosa/core/spectrum.py:222: UserWarning: n_fft=2048 is too small for input signal of length=1523
  warnings.warn(
8732it [06:56, 20.95it/s]


In [6]:
extracted_features_df=pd.DataFrame(extracted_features,columns=['feature','class'])
extracted_features_df.head()

,feature,class
0,"[-215.79301, 71.66612, -131.81377, -52.091328,...",dog_bark
1,"[-424.68677, 110.56227, -54.148235, 62.01073, ...",children_playing
2,"[-459.56467, 122.800354, -47.92471, 53.265694,...",children_playing
3,"[-414.55377, 102.896904, -36.66495, 54.18041, ...",children_playing
4,"[-447.397, 115.0954, -53.809113, 61.608585, 1....",children_playing


In [7]:
X=np.array(extracted_features_df['feature'].tolist())
y=np.array(extracted_features_df['class'].tolist())

In [8]:
labelencoder=LabelEncoder()
y=to_categorical(labelencoder.fit_transform(y))

In [9]:
X_train,X_test,y_train,y_test=train_test_split(X,y,test_size=0.2,random_state=0)

In [12]:
num_labels=y.shape[1]

In [13]:
# Model

model=Sequential()
###first layer
model.add(Dense(100,input_shape=(40,)))
model.add(Activation('relu'))
model.add(Dropout(0.5))
###second layer
model.add(Dense(200))
model.add(Activation('relu'))
model.add(Dropout(0.5))
###third layer
model.add(Dense(100))
model.add(Activation('relu'))
model.add(Dropout(0.5))

###final layer
model.add(Dense(num_labels))
model.add(Activation('softmax'))

model.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_3 (Dense)             (None, 100)               4100      
                                                                 
 activation_3 (Activation)   (None, 100)               0         
                                                                 
 dropout_3 (Dropout)         (None, 100)               0         
                                                                 
 dense_4 (Dense)             (None, 200)               20200     
                                                                 
 activation_4 (Activation)   (None, 200)               0         
                                                                 
 dropout_4 (Dropout)         (None, 200)               0         
                                                                 
 dense_5 (Dense)             (None, 100)              

In [14]:
model.compile(loss='categorical_crossentropy',metrics=['accuracy'],optimizer='adam')

In [15]:
num_epochs = 100
num_batch_size = 32

checkpointer = ModelCheckpoint(filepath='saved_models/audio_classification.hdf5', 
                               verbose=1, save_best_only=True)
start = datetime.now()

model.fit(X_train, y_train, batch_size=num_batch_size, epochs=num_epochs, validation_data=(X_test, y_test), callbacks=[checkpointer], verbose=1)

Epoch 1/100
216/219 [============================>.] - ETA: 0s - loss: 12.8285 - accuracy: 0.1141
Epoch 1: val_loss improved from inf to 2.29504, saving model to saved_models/audio_classification.hdf5
219/219 [==============================] - 1s 2ms/step - loss: 12.7220 - accuracy: 0.1144 - val_loss: 2.2950 - val_accuracy: 0.1070
Epoch 2/100
190/219 [=========================>....] - ETA: 0s - loss: 2.6427 - accuracy: 0.1130
Epoch 2: val_loss improved from 2.29504 to 2.28200, saving model to saved_models/audio_classification.hdf5
219/219 [==============================] - 0s 2ms/step - loss: 2.6147 - accuracy: 0.1150 - val_loss: 2.2820 - val_accuracy: 0.1070
Epoch 3/100
214/219 [============================>.] - ETA: 0s - loss: 2.3489 - accuracy: 0.1143
Epoch 3: val_loss improved from 2.28200 to 2.27510, saving model to saved_models/audio_classification.hdf5
219/219 [==============================] - 0s 2ms/step - loss: 2.3490 - accuracy: 0.1145 - val_loss: 2.2751 - val_accuracy: 0.10

219/219 [==============================] - 0s 2ms/step - loss: 1.7154 - accuracy: 0.3563 - val_loss: 1.5721 - val_accuracy: 0.4528
Epoch 26/100
180/219 [=======================>......] - ETA: 0s - loss: 1.6643 - accuracy: 0.3785
Epoch 26: val_loss improved from 1.57207 to 1.55814, saving model to saved_models/audio_classification.hdf5
219/219 [==============================] - 0s 1ms/step - loss: 1.6715 - accuracy: 0.3742 - val_loss: 1.5581 - val_accuracy: 0.4282
Epoch 27/100
186/219 [========================>.....] - ETA: 0s - loss: 1.6281 - accuracy: 0.3903
Epoch 27: val_loss improved from 1.55814 to 1.49919, saving model to saved_models/audio_classification.hdf5
219/219 [==============================] - 0s 1ms/step - loss: 1.6329 - accuracy: 0.3918 - val_loss: 1.4992 - val_accuracy: 0.4814
Epoch 28/100
187/219 [========================>.....] - ETA: 0s - loss: 1.6020 - accuracy: 0.3972
Epoch 28: val_loss improved from 1.49919 to 1.45715, saving model to saved_models/audio_classific

Epoch 51/100
179/219 [=======================>......] - ETA: 0s - loss: 1.1810 - accuracy: 0.5936
Epoch 51: val_loss did not improve from 0.98241
219/219 [==============================] - 0s 1ms/step - loss: 1.1737 - accuracy: 0.5984 - val_loss: 0.9878 - val_accuracy: 0.6800
Epoch 52/100
180/219 [=======================>......] - ETA: 0s - loss: 1.1791 - accuracy: 0.5993
Epoch 52: val_loss did not improve from 0.98241
219/219 [==============================] - 0s 1ms/step - loss: 1.1816 - accuracy: 0.6013 - val_loss: 1.0006 - val_accuracy: 0.6726
Epoch 53/100
219/219 [==============================] - ETA: 0s - loss: 1.1644 - accuracy: 0.6042
Epoch 53: val_loss improved from 0.98241 to 0.95789, saving model to saved_models/audio_classification.hdf5
219/219 [==============================] - 0s 2ms/step - loss: 1.1644 - accuracy: 0.6042 - val_loss: 0.9579 - val_accuracy: 0.7006
Epoch 54/100
215/219 [============================>.] - ETA: 0s - loss: 1.1221 - accuracy: 0.6240
Epoch 54: v

Epoch 78/100
186/219 [========================>.....] - ETA: 0s - loss: 0.9985 - accuracy: 0.6655
Epoch 78: val_loss improved from 0.83553 to 0.82161, saving model to saved_models/audio_classification.hdf5
219/219 [==============================] - 0s 1ms/step - loss: 0.9996 - accuracy: 0.6661 - val_loss: 0.8216 - val_accuracy: 0.7424
Epoch 79/100
187/219 [========================>.....] - ETA: 0s - loss: 1.0216 - accuracy: 0.6649
Epoch 79: val_loss did not improve from 0.82161
219/219 [==============================] - 0s 1ms/step - loss: 1.0178 - accuracy: 0.6667 - val_loss: 0.8250 - val_accuracy: 0.7424
Epoch 80/100
189/219 [========================>.....] - ETA: 0s - loss: 1.0021 - accuracy: 0.6673
Epoch 80: val_loss improved from 0.82161 to 0.81342, saving model to saved_models/audio_classification.hdf5
219/219 [==============================] - 0s 1ms/step - loss: 0.9971 - accuracy: 0.6674 - val_loss: 0.8134 - val_accuracy: 0.7436
Epoch 81/100
188/219 [========================>..

In [16]:
test_accuracy=model.evaluate(X_test,y_test,verbose=0)
print(test_accuracy[1])

0.7727532982826233


In [17]:
filename='UrbanSound8K/audio/fold3/6988-5-0-1.wav'
audio, sample_rate = librosa.load(filename, res_type='kaiser_fast') 
mfccs_features = librosa.feature.mfcc(y=audio, sr=sample_rate, n_mfcc=40)
mfccs_scaled_features = np.mean(mfccs_features.T,axis=0)

print(mfccs_scaled_features)
mfccs_scaled_features=mfccs_scaled_features.reshape(1,-1)
print(mfccs_scaled_features)
print(mfccs_scaled_features.shape)
predicted_label=model.predict(mfccs_scaled_features)
print(predicted_label)
prediction_class = np.argmax(predicted_label, axis=1)
print(label_dict[prediction_class[0]])

[-124.41564     162.64157     -33.432667     27.59999       6.6116724
   21.610598      2.9962502    19.552324      1.294804     17.197958
   -1.8212044    13.08889       3.2787871     3.667115      0.96994126
    7.140642     -0.7302965     4.0208635     3.9708393     3.6621003
    4.8365865     4.4332657     4.394522      2.5353334     8.956006
    3.222451      6.7038193     4.445663      6.2371874     4.125844
    3.1068814     1.4973228     1.8097677     0.9489547     2.7819495
    2.6311927     4.027545      1.5059777     3.3667514     3.2168236 ]
[[-124.41564     162.64157     -33.432667     27.59999       6.6116724
    21.610598      2.9962502    19.552324      1.294804     17.197958
    -1.8212044    13.08889       3.2787871     3.667115      0.96994126
     7.140642     -0.7302965     4.0208635     3.9708393     3.6621003
     4.8365865     4.4332657     4.394522      2.5353334     8.956006
     3.222451      6.7038193     4.445663      6.2371874     4.125844
     3.1068814  